# Purpose: 
- Preprocess and save the post-RTC image
- Partition the image into multiple segments and save the label as a np.array

In [8]:
import sys
from pathlib import Path

# Add the path to the utils folder to sys.path
utils_path = Path('../python').resolve()
sys.path.append(str(utils_path))

import rasterio
import numpy as np
from pathlib import Path
from process_utils import (preprocess_data,
                           superpixel_segmentation)
from rio_utils import (reproject_arr_to_match_profile)
from edit_path_utils import (edit_paths)

---
## Load the images into arrays

**Parameter**:
- `tifs` (list): a list containing the paths to the images

In [9]:
# Opens a GeoTIFF and loads the backscatter values and profile
def open_one(path):
    with rasterio.open(path) as ds:
        band = ds.read(1)
        profile = ds.profile
    return band, profile

In [10]:
# Path to the folder with the cropped images
data_dir = Path('/Volumes/BlackT7/bobcat_flight_paths')
tifs = sorted(list(data_dir.glob('./*26526_08525*.tif')))
# tifs = sorted([tif for tif in data_dir.glob('./*rtc_cropped*.tif') if 'preprocessed' not in tif.name])

# adjust for wanted and unwanted tifs
include = [
            "181011"
          ]
exclude = [
            "hello"
          ]

tifs = edit_paths(include,exclude,tifs)
print(*tifs, sep="\n")

[PosixPath('/Volumes/BlackT7/bobcat_flight_paths/bobcat_26526_08525_hv_0.tif'),
 PosixPath('/Volumes/BlackT7/bobcat_flight_paths/bobcat_26526_08525_hv_1.tif')]

---
Open the raster images and preprocess them.

Let `hv_0` and `profile_0` correspond to the pre-fire image, and `hv_1` and `profile_1` correspond to the post-fire image

In [12]:
# Open the tifs
bands, profiles = zip(*map(open_one, tifs))
hv_0 = bands[0]
hv_1 = bands[1]
profile_0 = profiles[0]
profile_1 = profiles[1]

---
## Preprocess the image
Applies interpolation, clipping, total-variation denoising, and background mask

**Parameters**
- `interpolation` (bool): whether or not to perform nearest neighbor interpolation for the preprocessing
- `weight` (float): denoising weight. The greater the weight, the more denoising (at the expense of fidelity to image). 

In [13]:
interpolation = False
weight = 5

In [14]:
hv_0 = preprocess_data(hv_0, interpolation, weight)
hv_1 = preprocess_data(hv_1, interpolation, weight)
print("Preprocessing done")

Preprocessing done


---
Reprojecting the array to match the profile of pre- and post- fire images, so arithmetic could be performed

In [15]:
# reproject the later flight to match the profile of the earlier flight
hv_1, _ = reproject_arr_to_match_profile(hv_1, profile_1, profile_0, resampling='bilinear')
hv_1 = hv_1[0]  # getting back to 2-D
print("Reproject done")

Reproject done


___
Saving the output to avoid long run-time in the future

**Parameters**
- `output_path_0` (str): output path for the processed pre-fire image [**.tif** file]
- `output_path_1` (str): output path for the processed post-fire image [**.tif** file]

In [16]:
output_path_0 = "/Volumes/BlackT7/hv_0_25km_preprocessed_interpolated_weight5.tif"
output_path_1 = "/Volumes/BlackT7/hv_1_25km_preprocessed_interpolated_weight5.tif"

In [17]:
with rasterio.open(output_path_0, "w", **profile_0) as dest:
    dest.write(hv_0, 1)
with rasterio.open(output_path_1, "w", **profile_0) as dest:
    dest.write(hv_1, 1)

___ 
## Superpixel Segmentation
Performs superpixel segmentation using [Felzenszwalb's algorithm](https://scikit-image.org/docs/stable/api/skimage.segmentation.html#skimage.segmentation.felzenszwalb) implemented by scikit-image to partition each image into multiple segments. The run time depends on the size of the images (~5-30mins)

**Parameters**
- `min_size` (int): minimum component size for Felzenszwalb's algorithm. Enforced using postprocessing. Check the hyperlink about scikit-learn implementation for more information
- `superpixel_out_path` (str): output path for the superpixel label as np.array. [**.npy**]

In [23]:
superpixel_labels = superpixel_segmentation(hv_0, hv_1, min_size=1000)
print("Superpixel done")

Superpixel done


In [24]:
superpixel_out_path = "/Volumes/BlueT7/test_bob/superpixel_labels_08525_08527_min1000_knear_weight10.npy"

In [25]:
np.save(superpixel_out_path, superpixel_labels)